In [1]:
import tensorflow as tf
import numpy as np

## 构建一个简单的线性模型

In [2]:
W = tf.Variable(1.,name="W")
b = tf.Variable(-1.,name="b")
x = tf.placeholder(tf.float32,name='x')
y =tf.placeholder(tf.float32,name='y')
y_ =tf.add(tf.multiply(W,x,name="W_x"),b,name="output")

input_x=np.array([1.1,2.03,2.95,4.08,5.05])
input_y=np.array([0.95,1.98,3.11,4.00,4.99])

loss = tf.reduce_sum(tf.square(y-y_))
optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

init = tf.global_variables_initializer()


## 使用tf.train.Saver

关于使用Saver保存的模型结构可以参考[A quick complete tutorial to save and restore Tensorflow models](http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/)

### 保存模型参数

In [5]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for _ in range(100):
        W_,b_,_=sess.run([W,b,optimizer],{x:input_x,y:input_y})
    print(W.eval(),b.eval())
    saver.save(sess,"savedModel/02/model.ckpt")

1.05747 -0.237715


### 读取模型参数
**注：这里必须重新运行kernel，否则会出现变量定义重复而出错**

In [2]:
W1 = tf.Variable(0.,name="W")
b1  = tf.Variable(-1.,name="b")
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,"savedModel/02/model.ckpt")
    print(W1.eval(),b1.eval())

INFO:tensorflow:Restoring parameters from savedModel/02/model.ckpt
1.05747 -0.237715
<class 'numpy.float32'>


## 使用[tf.train.write_graph()](https://www.tensorflow.org/api_docs/python/tf/train/write_graph)

### 保存模型

tensorflow模型保存的文件格式可参考[A Tool Developer's Guide to TensorFlow Model Files](https://www.tensorflow.org/extend/tool_developers/)，我们可以将模型保存为文本类型或二进制，通常二进制文件会更小，这代表在大模型的场景下，二进制文件更适用

这里要说明的是，tensorflow里的组件都有对应的def类，比如Graph有GraphDef类，这些Def是基于[Protocol Buffers](https://developers.google.com/protocol-buffers/?hl=en) 生成的。可以将GraphDef看做是Graph的元数据（定义），定义了Graph里的Node（Node也有对应的NodeDef）、version等（具体数据参考[GraphDef定义](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/graph.proto)）。我们实际保存的图模型也是GraphDef.

write_graph()函数其实也只是简单的将转换后的GraphDef序列化后保存到文件中，我们可以自行调用GraphDef.SerializeToString()获得序列化后的输出，再写入文件

In [15]:
with tf.Session() as sess:
    #训练模型
    sess.run(init)
    for _ in range(100):
        W_,b_,_=sess.run([W,b,optimizer],{x:input_x,y:input_y})
    
    #将模型中的变量（variables）全部变为constants，在生产环境下如果不需要继续训练模型而只使用训练好的模型的参数，则可以将所有的变量转换为常量
    #如果不转换为常量，则训练好的variables的值无法保存
    #可以减小模型的大小
    #这里返回的即为一个GraphDef类
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess , sess.graph_def , output_node_names = ["output"])
    #最后一个参数表示是否保存为文本文件格式，这里选择不用文本格式（二进制保存）
    tf.train.write_graph(output_graph,"savedModel/02","model1.pb",False)


NameError: name 'init' is not defined

### 读取模型

在读取模型的时候，也需要根据保存的类型来进行不同方式的读取，这里先以之前保存的二进制方式为例。读取时，我们需要创建一个新的GraphDef来加载保存的模型，然后使用tf.import_graph_def将加载好的模型引用到原模型中去

In [16]:
with tf.Session() as sess:
    #gfile为TF里的一个文件读取工具，我们直接使用它
    with tf.gfile.FastGFile("savedModel/02/model1.pb","rb") as gfile:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(gfile.read())
        g_in = tf.import_graph_def(graph_def)
    #可以通过这里查看所有的操作
    for ope in sess.graph.get_operations():
            print(ope)
    #注意变量的名字增加了前缀import/，这可以在调用tf.import_graph_def()是指定参数name="" 来更改     
    x = sess.graph.get_operation_by_name('import/x')
    #tensorflow中tensor的引用格式为 {operation_name}:{index}，表示由对应的操作产生的第index个输出，这里使用对应的输入输出名字来执行运行
    print(sess.run('import/output:0',{'import/x:0':1.}))
    



name: "import/W"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 1.057468056678772
    }
  }
}

name: "import/W/read"
op: "Identity"
input: "import/W"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@import/W"
    }
  }
}

name: "import/b"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: -0.23771463334560394
    }
  }
}

name: "import/b/read"
op: "Identity"
input: "import/b"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@import/b"
    }
  }
}

name: "import/x"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
    }
  }
}

name: "i